In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import math
import os
import copy

# konfigurasi
RANDOM_SEED = 42
NUM_STUDENTS = 1000
TOTAL_ITEMS = 240 
ITEMS_PER_LEVEL = 40
GUESSING_PARAM = 0.20 

np.random.seed(RANDOM_SEED)

LEVELS = [1, 2, 3, 4, 5, 6] 


# SL = Start Level
# TL = Transition Level (Streak Requirement)
# ST = Stability Threshold

START_LEVELS = {'SL1': 1, 'SL2': 2, 'SL3': 3}
TRANSITION_RULES = {'TL1': 1, 'TL2': 2, 'TL3': 3} 
STABILITY_THRESHOLDS = {'ST1': 1.5, 'ST2': 2.0, 'ST3': 2.5} 

# membuat siswa sintetis dengan rentang kemampuan dari -2.5 hingga +2.5, 
# dengan kategori rendah, sedang, tinggi
def generate_students(n=NUM_STUDENTS):
    thetas = np.random.normal(loc=0, scale=1, size=n)
    thetas = np.clip(thetas, -2.5, 2.5)
    
    students = []
    for i, theta in enumerate(thetas):
        if theta < -0.5:
            cat = 'Rendah'
        elif theta > 0.5:
            cat = 'Tinggi'
        else:
            cat = 'Sedang'
            
        students.append({
            'student_id': f'S{i+1:04d}',
            'true_ability': theta,
            'category': cat
        })
    
    return pd.DataFrame(students)

df_students = generate_students()
print(f"Students generated: {len(df_students)}")

# membuat bank soal dengan parameter a dan b yang berbeda untuk setiap 
# level kognitif, serta c tetap 0.20
def generate_item_bank():
    items = []
    level_params = {
        1: {'b_mean': -2.0, 'a_mean': 0.8},
        2: {'b_mean': -1.2, 'a_mean': 0.9},
        3: {'b_mean': -0.4, 'a_mean': 1.0},
        4: {'b_mean': 0.4, 'a_mean': 1.1},
        5: {'b_mean': 1.2, 'a_mean': 1.2},
        6: {'b_mean': 2.0, 'a_mean': 1.3}
    }
    
    item_counter = 1
    for lvl in LEVELS:
        for _ in range(ITEMS_PER_LEVEL):
            b = np.random.normal(level_params[lvl]['b_mean'], 0.4)
            a = np.random.normal(level_params[lvl]['a_mean'], 0.2)
            a = max(0.3, a) 
            
            items.append({
                'item_id': f'I{item_counter:03d}',
                'cognitive_level': lvl,
                'a': a,
                'b': b,
                'c': GUESSING_PARAM
            })
            item_counter += 1
            
    return pd.DataFrame(items)

df_items = generate_item_bank()
items_by_level = {lvl: df_items[df_items['cognitive_level'] == lvl].sample(frac=1, random_state=RANDOM_SEED).to_dict('records') for lvl in LEVELS}


# fungsi untuk menghitung probabilitas menjawab benar berdasarkan model 3PL
def get_probability_correct(theta, a, b, c):
    return c + (1 - c) / (1 + np.exp(-a * (theta - b)))
    # contoh perhitungan: theta=0.5, a=1.0, b=0.0, c=0.20
    # prob = 0.20 + (0.80 / (1 + exp(-1.0 * (0.5 - 0.0))))
    # prob = 0.20 + (0.80 / (1 + exp(-0.5)))
    # prob = 0.20 + (0.80 / (1 + 0.6065))
    # prob = 0.20 + (0.80 / 1.6065)
    # prob = 0.20 + 0.4979
    # prob = 0.6979
    # sehingga siswa dengan theta 0.5 memiliki probabilitas sekitar 
    # 69.79% untuk menjawab benar pada item dengan a=1.0, b=0.0, c=0.20

# fungsi untuk menghitung skor berdasarkan history jawaban siswa, 
# dengan bobot level kognitif
def calculate_score(hist):
    if not hist:
        return 0.0
    total_weight = sum(h['level'] for h in hist)
    if total_weight == 0:
        return 0.0

    correct_weight = sum(h['level'] for h in hist if h['response'] == 1)

    # Skor dihitung sebagai persentase bobot jawaban benar 
    # terhadap total bobot, lalu dikalibrasi ke skala 0-100
    return (correct_weight / total_weight) * 100

# fungsi utama untuk mensimulasikan 1 sesi tes untuk 1 siswa
def simulate_single_test(student, start_lvl, streak_req, stab_thresh, item_pool_copy):
    theta = student['true_ability']
    
    # siswa memulai dari level awal yang ditentukan
    current_level = start_lvl
    
    # untuk mekanisme transisi, streak benar dan salah  perlu dihitung secara terpisah
    current_streak_correct = 0 
    current_streak_wrong = 0
    
    history = [] 
    
    is_terminated = False
    termination_reason = ""
    
    while not is_terminated:
        # memilih soal dari level saat ini
        if not item_pool_copy[current_level]:
            # Fallback: jika soal habis di level ini, coba cari di level terdekat
            break
            
        item = item_pool_copy[current_level].pop(0)
        
        # mensimulasikan respons siswa berdasarkan probabilitas benar dari model 3PL
        prob_correct = get_probability_correct(theta, item['a'], item['b'], item['c'])
        rand_val = np.random.uniform(0, 1)
        response = 1 if rand_val < prob_correct else 0
        
        # mengupdate log sementara untuk item terkini, termasuk level, parameter b, 
        # respons, dan skor setelah menjawab item terkini
        temp_log = {
            'item_id': item['item_id'],
            'level': current_level,
            'b': item['b'],
            'response': response
        }
        
        # menghitung skor setelah item terjawab, dengan memasukkan 
        # log sementara ke dalam history untuk perhitungan skor yang akurat
        current_score_val = calculate_score(history + [temp_log])
        temp_log['current_score'] = current_score_val
        history.append(temp_log)
        
        n_items = len(history)
        
        # mengecek aturan terminasi setelah setiap item dijawab, d
        # engan prioritas: batas maksimum soal, stabilitas skor, dan penguasaan level tertinggi (C6)
        
        # maksimum 40 soal untuk mencegah tes yang terlalu panjang
        if n_items >= 40:
            is_terminated = True
            termination_reason = "Max Items"
        
        # stabilitas skor: jika setelah 15 item, skor tidak berubah 
        # signifikan selama 5 item terakhir, tes dihentikan karena dianggap sudah stabil
        elif n_items >= 15:
            score_now = history[-1]['current_score']
            score_prev = history[-6]['current_score']
            
            if abs(score_now - score_prev) <= stab_thresh:
                is_terminated = True
                termination_reason = "Stability"
        
        # jika siswa berada di level 6, menjawab benar, dan memenuhi syarat streak, 
        # tes dihentikan karena dianggap sudah menguasai level tertinggi
        if not is_terminated and current_level == 6 and response == 1:
            # cek streak benar untuk menentukan apakah siswa sudah cukup konsisten 
            # menjawab benar di level 6 untuk dianggap menguasai
            if (current_streak_correct + 1) >= streak_req:
                is_terminated = True
                termination_reason = "C6 Mastery"

        # transisi level dilakukan setelah mengecek terminasi, sehingga jika 
        # siswa memenuhi syarat untuk naik atau turun level, transisi tetap terjadi sebelum cek terminasi berikutnya
        if not is_terminated:
            if response == 1:
                # jika jawaban benar, tambahkan ke streak benar dan reset streak salah karena rantai putus
                current_streak_correct += 1
                current_streak_wrong = 0  # reset streak salah karena jawaban benar memutus rantai salah
                
                # mengecek apakah memenuhi syarat naik Level
                if current_streak_correct >= streak_req:
                    if current_level < 6:
                        current_level += 1
                        current_streak_correct = 0 # reset streak setelah naik
                    else:
                        pass # sudah di level max, pertahankan level terkini
            else:
                # jika jawaban salah, tambahkan ke streak salah dan reset streak benar karena rantai putus
                current_streak_wrong += 1
                current_streak_correct = 0 # reset streak benar karena jawaban salah memutus rantai benar
                
                # mengecek apakah memenuhi syarat turun Level
                if current_streak_wrong >= streak_req:
                    if current_level > 1:
                        current_level -= 1
                        current_streak_wrong = 0 # reset streak setelah turun
                    else:
                        pass # sudah di level min, pertahankan level terkini

    # final calculation setelah tes dihentikan, termasuk skor akhir, akurasi, dan alasan terminasi
    final_score = history[-1]['current_score'] if history else 0.0
    total_correct = sum([h['response'] for h in history])
    accuracy = (total_correct / len(history)) * 100 if history else 0.0
    
    return {
        'final_score': final_score,
        'accuracy': accuracy,
        'history': history,
        'total_items': len(history),
        'total_correct': total_correct,
        'reason': termination_reason
    }
    

# membuat skenario dengan kombinasi dari Start Level (SL), Transition Level (TL), 
# dan Stability Threshold (ST), sehingga total skenario menjadi 3 x 3 x 3 = 27
scenarios = []
# update loop untuk menggunakan Key baru (SL, TL, ST)
for sl_name, sl_val in START_LEVELS.items():
    for tl_name, tl_val in TRANSITION_RULES.items():
        for st_name, st_val in STABILITY_THRESHOLDS.items():
            # membuat id skenario unik berdasarkan kombinasi konfigurasi skenario
            sc_id = f"{sl_name}-{tl_name}-{st_name}" 
            scenarios.append({
                'id': sc_id,
                'start_lvl': sl_val,
                'streak_req': tl_val,
                'stab_thresh': st_val
            })

print(f"Total Skenario: {len(scenarios)}")

summary_results = []
scenario_logs_data = {} 
item_logs_data = [] 

print("Mulai Simulasi... (Ini mungkin memakan waktu)")

for sc in scenarios:
    sc_id = sc['id']
    sc_student_results = []
    
    items_low, items_med, items_high = [], [], []
    pred_scores = []
    true_thetas = []
    biases = []
    
    for _, student in df_students.iterrows():
        current_pool = copy.deepcopy(items_by_level)
        
        result = simulate_single_test(
            student, 
            sc['start_lvl'], 
            sc['streak_req'], 
            sc['stab_thresh'], 
            current_pool
        )
        
        sc_student_results.append({
            'student_id': student['student_id'],
            'true_ability': student['true_ability'],
            'ability_category': student['category'],
            'final_estimation_score': result['final_score'],
            'accuracy': result['accuracy'],
            'stability_threshold': sc['stab_thresh'],
            'correct_answers': result['total_correct'],
            'total_items': result['total_items'],
            'termination_reason': result['reason']
        })
        
        for log in result['history']:
            item_logs_data.append({
                'scenario_id': sc_id,
                'student_id': student['student_id'],
                'item_id': log['item_id'],
                'cognitive_level': log['level'],
                'item_b': log['b'],
                'response': log['response'],
                'estimation_score_after_item': log['current_score']
            })
            
        pred_scores.append(result['final_score'])
        true_thetas.append(student['true_ability'])
        
        # Bias: Score (0-100) dikurang Theta (-2.5 s.d 2.5)
        theta_proj = (student['true_ability'] + 2.5) / 5 * 100
        biases.append(result['final_score'] - theta_proj) 
        
        if student['category'] == 'Rendah':
            items_low.append(result['total_items'])
        elif student['category'] == 'Sedang':
            items_med.append(result['total_items'])
        elif student['category'] == 'Tinggi':
            items_high.append(result['total_items'])

    # mengkalkulasi metrik evaluasi untuk skenario ini, termasuk korelasi Spearman dan Pearson
    spearman_corr, _ = stats.spearmanr(true_thetas, pred_scores)
    pearson_corr, _ = stats.pearsonr(true_thetas, pred_scores)
    mean_bias = np.mean(biases)
    
    avg_items_low = np.mean(items_low) if items_low else 0
    avg_items_med = np.mean(items_med) if items_med else 0
    avg_items_high = np.mean(items_high) if items_high else 0
    avg_items_total = np.mean(items_low + items_med + items_high)
    
    summary_results.append({
        'Skenario': sc_id,
        'Spearman': spearman_corr,
        'Pearson': pearson_corr,
        'Bias': mean_bias,
        'Avg Soal (Rendah)': avg_items_low,
        'Avg Soal (Sedang)': avg_items_med,
        'Avg Soal (Tinggi)': avg_items_high,
        'Avg Soal': avg_items_total
    })
    
    scenario_logs_data[sc_id] = pd.DataFrame(sc_student_results)

print("Simulasi Selesai.")


# menyimpan hasil simulasi ke file CSV dan Excel untuk analisis lebih lanjut
df_summary = pd.DataFrame(summary_results)
df_summary.sort_values(by='Spearman', ascending=False, inplace=True)
df_summary.to_csv('summary_simulation_results.csv', index=False)
print("Saved: summary_simulation_results.csv")

with pd.ExcelWriter('scenario_level_logs.xlsx') as writer:
    for sc_id, df_sc in scenario_logs_data.items():
        sheet_name = sc_id[:31] 
        df_sc.to_excel(writer, sheet_name=sheet_name, index=False)
print("Saved: scenario_level_logs.xlsx")

df_item_logs = pd.DataFrame(item_logs_data)
df_item_logs.to_csv('item_level_logs.csv', index=False)
print("Saved: item_level_logs.csv")


# preview
print("\n--- Konfigurasi Terbaik  ---")
print(df_summary[['Skenario', 'Spearman', 'Bias', 'Avg Soal']].head(3))

Students generated: 1000
Total Skenario: 27
Mulai Simulasi... (Ini mungkin memakan waktu)
Simulasi Selesai.
Saved: summary_simulation_results.csv
Saved: scenario_level_logs.xlsx
Saved: item_level_logs.csv

--- Konfigurasi Terbaik  ---
       Skenario  Spearman      Bias  Avg Soal
23  SL3-TL2-ST3  0.764042  3.093633    17.412
22  SL3-TL2-ST2  0.749735  3.417670    18.353
21  SL3-TL2-ST1  0.743532  3.324278    20.032
